## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten, Dropout, Dense
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50
import numpy as np
import os

#gpu_devices = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpu_devices[0], True)

In [10]:
# Training parameters

# 資料路徑
DATASET_PATH  = '/content'

# 影像大小
IMAGE_SIZE = (224, 224)

# 影像類別數
NUM_CLASSES = 10

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 128

# Epoch 數
NUM_EPOCHS = 50

data_augmentation = True

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

In [11]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [12]:
#initial input shape
input_shape = x_train.shape[1:]

# Normalize
x_train = x_train/255
x_test = x_test/255

#one-hot coding label
y_train =  keras.utils.to_categorical(y_train,NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test,NUM_CLASSES)

if subtract_pixel_mean:
  x_train_mean = np.mean(x_train)
  x_test_mean = np.mean(x_test)
  x_train = x_train - x_train_mean
  x_test = x_test - x_test_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 10)


In [13]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 40:
        lr *= 1e-3
    elif epoch > 30:
        lr *= 1e-2
    elif epoch > 20:
        lr *= 1e-1
    elif epoch > 10:
        lr *= 3e-1
    print('Learning rate: ', lr)
    return lr

In [14]:
class Mycallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print(K.eval(self.model.optimizer.lr))

In [21]:
# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath="/content",
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=3,
                               min_lr=1e-8)

callbacks = [checkpoint, lr_reducer, lr_scheduler, Mycallback()]

In [22]:
# initialize pre-trained model
net = ResNet50(include_top=False, 
               weights='imagenet', 
               input_tensor=None,
               input_shape=(32, 32, 3))


#remove classification layer
net.layers.pop()
net = Model(inputs=net.input, outputs=net.layers[-1].output)

for layer in net.layers[:-5]:
    layer.trainable = False

x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', kernel_initializer='he_normal', name="classifier")(x)

net_final = Model(inputs=net.input, outputs=output_layer)

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=RMSprop(learning_rate=lr_schedule(0)),
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

Learning rate:  0.001
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________

In [23]:
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    net_final.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    # 透過 data augmentation 產生訓練與驗證用的影像資料
    net_final.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
    
    train_datagen = ImageDataGenerator(rotation_range=20,
                                       width_shift_range=0.1,
                                       height_shift_range=0.1,
                                       shear_range=0.1,
                                       zoom_range=0.1,
                                       channel_shift_range=0,
                                       horizontal_flip=True,
                                       fill_mode='nearest')
    model_value_aug = net_final.fit_generator(train_datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                                  validation_data=(x_test, y_test), 
                                  steps_per_epoch=len(x_train) // BATCH_SIZE,
                                  epochs=NUM_EPOCHS,
                                  callbacks=callbacks)


Using real-time data augmentation.
Epoch 1/50
Learning rate:  0.001
391/391 [==============================] - 15s 31ms/step - loss: 2.3503 - accuracy: 0.2710 - val_loss: 5.0663 - val_accuracy: 0.1802

Epoch 00001: val_accuracy improved from -inf to 0.18020, saving model to /content
INFO:tensorflow:Assets written to: /content/assets
0.001
Epoch 2/50
Learning rate:  0.001
391/391 [==============================] - 11s 29ms/step - loss: 1.7194 - accuracy: 0.3861 - val_loss: 13.2861 - val_accuracy: 0.1327

Epoch 00002: val_accuracy did not improve from 0.18020
0.001
Epoch 3/50
Learning rate:  0.001
391/391 [==============================] - 11s 29ms/step - loss: 1.6502 - accuracy: 0.4158 - val_loss: 9.2066 - val_accuracy: 0.2430

Epoch 00003: val_accuracy improved from 0.18020 to 0.24300, saving model to /content
INFO:tensorflow:Assets written to: /content/assets
0.001
Epoch 4/50
Learning rate:  0.001
391/391 [==============================] - 11s 29ms/step - loss: 1.5984 - accuracy: 0.43

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
Learning rate:  0.001
390/390 [==============================] - 34s 82ms/step - loss: 1.6613 - accuracy: 0.4191 - val_loss: 231.2759 - val_accuracy: 0.1662

Epoch 00001: val_accuracy did not improve from 0.53390
0.001
Epoch 2/50
Learning rate:  0.001
390/390 [==============================] - 32s 81ms/step - loss: 1.6574 - accuracy: 0.4166 - val_loss: 167.5126 - val_accuracy: 0.1716

Epoch 00002: val_accuracy did not improve from 0.53390
0.001
Epoch 3/50
Learning rate:  0.001
390/390 [==============================] - 31s 81ms/step - loss: 1.6537 - accuracy: 0.4183 - val_loss: 133.5513 - val_accuracy: 0.1802

Epoch 00003: val_accuracy did not improve from 0.53390
0.001
Epoch 4/50
Learning rate:  0.001
390/390 [==============================] - 32s 81ms/step - loss: 1.6367 - accuracy: 0.4237 - val_loss: 137.4284 - val_accuracy: 0.1564

Epoch 00004: val_accuracy did not improve from 0.53390
0.001
Epoch 5/50
Learning rate:  0.001
390/390 [==============================] - 32s 

In [25]:
# Score trained model.
scores = net_final.evaluate(x_test,y_test,verbose=1)
print('Test loss:',scores[0])
print('Test accuracy:',scores[1])

313/313 [==============================] - 4s 12ms/step - loss: 1.3453 - accuracy: 0.5342
Test loss: 1.3452526330947876
Test accuracy: 0.5342000126838684
